In [1]:
# blackjack.py
# I made this because I'm convinced that blackjack apps which have a monetization
# scheme will play unfairly in order to get players to put USD into the game.

import random, os, sys, csv, math, statistics, time

auto_play = False
auto_play_games = 50
auto_wager = 2

filename = 'blackjack_outcomes.csv'
file_exists = os.path.isfile(filename)
fieldnames = ['games played',
              'player wins',
              'player blackjacks',
              'house wins',
              'house blackjacks',
              'draws count',
              'player chips']

num_games = 0
player_wins_count = 0
house_wins_count = 0
draws_count = 0
player_blackjack_count = 0
house_blackjack_count = 0
this_session_games = 0
player_stack = 100


# Open database containing play statistics and persistent player stack
if file_exists:
    with open(filename, 'r') as f:
        csv_reader = csv.DictReader(f)
        csv_data = list(csv_reader)

    for line in csv_data:
        num_games = int(line['games played'])
        player_wins_count = int(line['player wins'])
        house_wins_count = int(line['house wins'])
        draws_count = int(line['draws count'])
        player_blackjack_count = int(line['player blackjacks'])
        house_blackjack_count = int(line['house blackjacks'])
        player_stack = int(line['player chips'])


class Card:
    def __init__(self, rank, suit, sets):
        self.rank = rank
        self.suit = suit
        self.sets = sets
        if rank in range(1, 14):
            self.value = rank
        if rank in ['J', 'Q', 'K']:
            self.value = 10
        if rank == 'A':
            self.value = 11

        # This is the "shuffle card" which when drawn indicates that the end of the
        # deck is coming up and a new deck is needed.
        if rank == 0:
            self.rank = suit
            self.suit = suit
            self.value = suit

    def __str__(self):
        return '%s of %s' % (self.rank, self.suit)


class Hand:
    def __init__(self, player, hand_number):
        self.player = player
        self.cards = []
        self.value = 0
        self.ace_count = 0
        self.soft_hand = False
        self.hand_number = hand_number

    def hand_value(self):
        self.value = 0
        self.ace_count = 0
        for card in self.cards:
            self.value += card.value
        for card in self.cards:
            if card.value == 11:
                self.ace_count += 1
                self.soft_hand = True
        while self.value > 21 and self.ace_count > 0:
            print('reducing ace')
            self.ace_count -= 1
            self.value -= 10
            self.soft_hand = False
        return self.value

    def __str__(self):
        print('%s\'s hand:' % self.player)
        for card in self.cards:
            print(card)


suits = ["Clubs", "Spades", "Hearts", "Diamonds"]
ranks = ['A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K']

deck = []
num_decks = 8
num_shuffles = 3
current_wager = 0
num_hands = 1

continue_play = True
player_turn = True
dealer_turn = False
shuffle_needed = True
first_game = True
end_turn = False


def shuffle_deck():
    # Each time shuffle_deck() is called, the current deck is overwritten by num_decks new 52
    # card decks and then shuffled num_shuffles times.
    deck = []
    global shuffle_needed
    for sets in range(num_decks):
        for rank in ranks:
            for suit in suits:
                deck.append(Card(rank, suit, sets))

    if num_shuffles == 1:
        for shuffle in range(num_shuffles - 1, -1, -1):
            print('Shuffling %s deck one time. Total of %s cards' % (num_decks, len(deck)))
            for i in range(len(deck) - 1, 0, -1):
                j = random.randint(0, i)
                deck[i], deck[j] = deck[j], deck[i]
    else:
        for shuffle in range(num_shuffles - 1, -1, -1):
            print('Shuffling %s decks %s more times. Total of %s cards' % (num_decks, shuffle, len(deck)))
            for i in range(len(deck) - 1, 0, -1):
                j = random.randint(0, i)
                deck[i], deck[j] = deck[j], deck[i]

    offsetbound = round(len(deck) * .1)
    shuffleoffset = random.randint(-offsetbound, offsetbound)
    shufflespot = round(len(deck) * 0.85) + shuffleoffset
    deck.insert(shufflespot - 5, Card(0, 'shuffle', 'shuffle'))

    shuffle_needed = False
    return deck


def set_wager(wager_amount):
    # set_wager() takes the current chips on the table, adds then to the player stack
    # then removes wager_amount chips from the player stack and puts them on the table.
    global player_stack
    global current_wager
    try:
        number = int(wager_amount)
    except ValueError:
        print('Wager amount must be a whole number of chips, not %s' % wager_amount)
        print('I\'ll make your wager the minimum of 2 chips')
        wager_amount = 2
        player_stack += current_wager
        player_stack -= wager_amount
        current_wager = wager_amount
        print('Current wager: %s' % current_wager)
        return current_wager
    else:
        wager_amount = int(wager_amount)
        if wager_amount < 2:
            print('Minimum wager is 2 chips, I\'ll set your wager to 2 chips.')
            wager_amount = 2
            player_stack += current_wager
            player_stack -= wager_amount
            current_wager = wager_amount
        else:
            player_stack += current_wager
            player_stack -= wager_amount
            current_wager = wager_amount
        print('Current wager: %s' % current_wager)
        return current_wager


def player_wins():
    print('\n\nPlayer Wins. House pays 2:1\n\n')
    global player_wins_count
    global player_stack
    player_wins_count += 1
    player_stack += current_wager
    print('Player wins %s chips' % current_wager)
    if double_down:
        player_stack += current_wager // 2
    return player_wins_count


def player_blackjack():
    print('\n\nPlayer Wins. Black Jack pays 3:2\n\n')
    global player_blackjack_count
    global player_stack
    player_blackjack_count += 1
    player_stack += int(3/2 * current_wager)
    print('Player wins %s chips' % (int(math.ceil(3/2 * current_wager))))
    return player_blackjack_count


def dealer_wins():
    print('\n\nHouse Wins. House takes bet.\n\n')
    global house_wins_count
    global player_stack
    house_wins_count += 1
    if double_down:
        player_stack -= current_wager // 2
    else:
        player_stack -= current_wager
    print('Player lost %s chips' % current_wager)
    return house_wins_count


def dealer_blackjack():
    print('\n\nDealer has blackjack. House takes bet.\n\n')
    global house_blackjack_count
    global player_stack
    house_blackjack_count += 1
    player_stack -= current_wager
    return house_blackjack_count


def push():
    print('\n\nPush. Player keeps bet.\n\n')
    global draws_count
    draws_count += 1
    return draws_count


def blackjack_push():
    print('\n\nPush with BlackJack?!? Terrible luck...\n\n')
    global draws_count
    draws_count += 1
    return draws_count


def deal_card(where):
    # deal_card() looks for the shuffle card, then pops the first card out of the deck
    # and appends it to the where hand.
    global shuffle_needed
    if deck[0].rank == 'shuffle':
        print('_,.-.,__,.-.,_ The shuffle card has been reached _,.-.,__,.-.,_')
        shuffle_needed = True
        deck.pop(0)
    where.cards.append(deck.pop(0))
    where.hand_value()
    print('A card was dealt and now there are %s cards left in the deck\n' % len(deck))


def show_hand(hand):
    # show_hand() prints hand using the __str__ method in Card
    print('%s %s\'s hand:' % (hand.player, hand.hand_number))
    for card in hand.cards:
        print(card)


def show_chips():
    print('Player\'s Stack: %s' % player_stack)


def hand_splitter():
    # hand_splitter() removes one card from the current hand and puts it into a new hand.
    global num_hands
    split_player_hand = Hand(player_hand[0].player, num_hands)
    split_player_hand.cards.append(player_hand[num_hands - 1].cards.pop(0))
    deal_card(player_hand[num_hands - 1])
    player_hand.append(split_player_hand)
    num_hands += 1


def write_out():
    # Writes the game statistics and persistant player stack out to database.
    line = {'games played': num_games,
            'player wins': player_wins_count,
            'player blackjacks': player_blackjack_count,
            'house wins': house_wins_count,
            'house blackjacks': house_blackjack_count,
            'draws count': draws_count,
            'player chips': player_stack
            }

    with open(filename, 'a') as f:
        csv_writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:
            csv_writer.writeheader()
        csv_writer.writerow(line)


Shuffling 8 decks 2 more times. Total of 416 cards
Shuffling 8 decks 1 more times. Total of 416 cards
Shuffling 8 decks 0 more times. Total of 416 cards


Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
Dealer Must Hit on Soft 17.
Player's Stack: -51239
Wager amount:   100
--- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
Current wager: 100
Player's Stack: -51339
A card was dealt and now there are 416 cards left in the deck

A card was dealt and now there are 415 cards left in the deck

A card was dealt and now there are 414 cards left in the deck

A card was dealt and now there are 413 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 412 cards left in the deck

A card was dealt and now there are 411 cards left in the deck

A card was dealt and now there are 410 cards left in the deck

A card was dealt and now there are 409 cards left in the deck

A card was dealt and now there are 222 cards left in the deck

A card was dealt and now there are 221 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 220 cards left in the deck

A card was dealt and now there are 219 cards left in the deck

A card was dealt and now there are 218 cards left in the deck

A card was dealt and now there are 217 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 216 cards left in the deck

A card was dealt and now there are 215 cards left in the deck

A card was dealt and now there are 214 cards left in the deck

A card was dealt and now there are 213 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 212 cards left in the deck

A card was dealt and now there are 211 cards left in the deck

A card was dealt




Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 132 cards left in the deck

A card was dealt and now there are 131 cards left in the deck

A card was dealt and now there are 130 cards left in the deck

A card was dealt and now there are 129 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 128 cards left in the deck

A card was dealt and now there are 127 cards left in the deck

A card was dealt and now there are 126 cards left in the deck

A card was dealt and now there are 125 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 124 cards left in the deck

A card was dealt and now there are 123 cards left in the deck

A card was dealt and now there are 122 cards left in the deck

A card was dealt and now there are 121 cards left in the deck



Total Player 



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 396 cards left in the deck

A card was dealt and now there are 395 cards left in the deck

A card was dealt and now there are 394 cards left in the deck

A card was dealt and now there are 393 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 392 cards left in the deck

A card was dealt and now there are 391 cards left in the deck

A card was dealt and now there are 390 cards left in the deck

A card was dealt and now there are 389 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 388 cards left in the deck

A card was dealt and now there are 387 cards left in the deck

A card was dealt and now there are 386 cards left in the deck

A card was dealt and now there are 385 cards left in the deck



Total Player W


A card was dealt and now there are 289 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 288 cards left in the deck

A card was dealt and now there are 287 cards left in the deck

A card was dealt and now there are 286 cards left in the deck

A card was dealt and now there are 285 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 284 cards left in the deck

A card was dealt and now there are 283 cards left in the deck

A card was dealt and now there are 282 cards left in the deck

A card was dealt and now there are 281 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 280 cards left in the deck

A card was dealt and now there are 279 cards left in the deck

A card was dealt and now there are 278 cards left in the deck

A card was deal

...

A card was dealt and now there are 140 cards left in the deck

A card was dealt and now there are 139 cards left in the deck

A card was dealt and now there are 138 cards left in the deck

A card was dealt and now there are 137 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 136 cards left in the deck

A card was dealt and now there are 135 cards left in the deck

A card was dealt and now there are 134 cards left in the deck

A card was dealt and now there are 133 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 132 cards left in the deck

A card was dealt and now there are 131 cards left in the deck

A card was dealt and now there are 130 cards left in the deck

A card was dealt and now there are 129 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt




Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 76 cards left in the deck

A card was dealt and now there are 75 cards left in the deck

A card was dealt and now there are 74 cards left in the deck

A card was dealt and now there are 73 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 72 cards left in the deck

A card was dealt and now there are 71 cards left in the deck

A card was dealt and now there are 70 cards left in the deck

A card was dealt and now there are 69 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 68 cards left in the deck

A card was dealt and now there are 67 cards left in the deck

A card was dealt and now there are 66 cards left in the deck

A card was dealt and now there are 65 cards left in the deck



Total Player Wins: 216600

A card was dealt and now there are 60 cards left in the deck

Shuffling 8 decks 2 more times. Total of 416 cards
Shuffling 8 decks 1 more times. Total of 416 cards
Shuffling 8 decks 0 more times. Total of 416 cards


Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 416 cards left in the deck

A card was dealt and now there are 415 cards left in the deck

A card was dealt and now there are 414 cards left in the deck

A card was dealt and now there are 413 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 412 cards left in the deck

A card was dealt and now there are 411 cards left in the deck

A card was dealt and now there are 410 cards left in the deck

A card was dealt and now there are 409 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 408 cards left in 


A card was dealt and now there are 331 cards left in the deck

A card was dealt and now there are 330 cards left in the deck

A card was dealt and now there are 329 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 328 cards left in the deck

A card was dealt and now there are 327 cards left in the deck

A card was dealt and now there are 326 cards left in the deck

A card was dealt and now there are 325 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 324 cards left in the deck

A card was dealt and now there are 323 cards left in the deck

A card was dealt and now there are 322 cards left in the deck

A card was dealt and now there are 321 cards left in the deck



Total Player Wins: 216600   Total House Wins: 256636   Total Draws: 47730
A card was dealt and now there are 320 cards left in the deck

A card was deal

KeyboardInterrupt: 

In [6]:
results = [{'Hand Number': 0, 'Hand Value': 5, 'Wager': 100}, {'Hand Number': 1, 'Hand Value': 7, 'Wager': 100}]

In [8]:
results[1].get('Hand Number')

1